# THE NETWORK OF VOLTAIRE'S PHILOSOPHICAL DICTIONARY 
## Notebook 1: Deriving the directed graph of references among dictionary items
## By M.A. Boudourides, H. Bin Fahad, Y. Malla, & S. Silwal

In [16]:
!pip3 install urllib2
# !conda install urllib2
# import inflect
# Change the following two lines:
# import os
# os.chdir("/Users/[User Name]/Desktop/")

import pandas as pd
import numpy as np
import urllib2 
import re, string, math, random, operator
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag
from textblob import TextBlob
import codecs

import pickle
from collections import Counter
import itertools as it
import networkx as nx
from networkx.algorithms import bipartite
from networkx.drawing.nx_agraph import graphviz_layout
import pygraphviz
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2


ModuleNotFoundError: No module named 'urllib2'

### List of Terms (manually collected)

In [ ]:
oldterms=sorted([
    "ADULTERY", 
    "ADVOCATE", 
    "ANCIENTS AND MODERNS", 
    "ANIMALS",
    "ANTIQUITY",
    "ARTS",
    "ASTROLOGY",
    "ATHEISM",
    "AUTHORITY",
    "AUTHORS",
    "BANISHMENT",
    "BANKRUPTCY",
    "BEAUTY",
    "BISHOP",
    "BOOKS",
    "BOULEVERD OR BOULEVART",
    "BOURGES",
    "BRAHMINS",
    "CHARACTER",
    "CHARLATAN",
    "CIVIL LAWS",
    "CLIMATE",
    "COMMON SENSE",
    "CONCATENATION OF EVENTS",
    "CONTRADICTIONS",
    "CORN",
    "CROMWELL",
    "CUSTOMS",
    "DEMOCRACY",
    "DESTINY",
    "DEVOUT",
    "THE ECCLESIASTICAL MINISTRY",
    "EMBLEM",
    "ON THE ENGLISH THEATRE",
    "ENVY",
    "EQUALITY",
    "EXPIATION",
    "EXTREME",
    "EZOURVEIDAM",
    "FAITH",
    "FALSE MINDS",
    "FATHERLAND",
    "FINAL CAUSES",
    "FRAUD",
    "FREE-WILL",
    "FRENCH",
    "FRIENDSHIP",
    "GOD",
    "HELVETIA",
    "HISTORY",
    "IGNORANCE",
    "THE IMPIOUS",
    "JOAN OF ARC",
    "KISSING",
    "LANGUAGES",
    "LAWS",
    "LIBERTY",
    "LIBRARY",
    "LIMITS OF THE HUMAN MIND",
    "LOCAL CRIMES",
    "LOVE",
    "LUXURY",
    "GENERAL REFLECTION ON MAN",
    "MAN IN THE IRON MASK",
    "MARRIAGE",
    "MASTER",
    "MEN OF LETTERS",
    "METAMORPHOSIS",
    "MILTON, ON THE REPROACH OF PLAGIARISM AGAINST",
    "MOHAMMEDANS",
    "MOUNTAIN",
    "NAKEDNESS",
    "NATURAL LAW",
    "NATURE",
    "NECESSARY",
    "NEW NOVELTIES",
    "PHILOSOPHER",
    "POWER",
    "OMNIPOTENCE",
    "PRAYERS",
    "PRÉCIS OF ANCIENT PHILOSOPHY",
    "PREJUDICES",
    "RARE",
    "REASON",
    "RELIGION",
    "SECT",
    "SELF-ESTEEM",
    "SOUL",
    "STATES",
    "GOVERNMENTS",
    "SUPERSTITION",
    "TEARS",
    "THEIST",
    "TOLERANCE",
    "TRUTH",
    "TYRANNY",
    "VIRTUE",
    "WHY?",
    ])
print len(oldterms)
# oldterms

In [ ]:
# terms=sorted([
#     'ADULTERY',
#  'ADVOCATE',
#  'ANIMAL',
#  'ANTIQUITY',
#  'ART',
#  'ASTROLOGY',
#  'ATHEISM',
#  'AUTHORITY',
#  'AUTHOR',
#  'BANISHMENT',
#  'BANKRUPTCY',
#  'BEAUTY',
#  'BISHOP',
#  'BOOK',
#  'BOURGES',
#  'BRAHMIN',
#  'CHARACTER',
#  'CHARLATAN',
#  'CLIMATE',
#  'CONTRADICTION',
#  'CORN',
#  'CROMWELL',
#  'CUSTOM',
#  'DEMOCRACY',
#  'DESTINY',
#  'EMBLEM',
#  'ENVY',
#  'EQUALITY',
#  'EXPIATION',
#  'EXTREME',
#  'EZOURVEIDAM',
#  'FAITH',
#  'FATHERLAND',
#  'FRAUD',
#  'FREE-WILL',
#  'FRENCH',
#  'FRIENDSHIP',
#  'GOD',
#  'GOVERNMENT',
#  'HELVETIA',
#  'HISTORY',
#  'IGNORANCE',
#  'KISS',
#  'LANGUAGE',
#  'LAW',
#  'LIBERTY',
#  'LIBRARY',
#  'LOVE',
#  'LUXURY',
#  'MARRIAGE',
#  'MASTER',
#  'METAMORPHOSIS',
#  'MOHAMMED',
#  'MOUNTAIN',
#  'NAKEDNESS',
#  'NATURE',
#  'NECESSARY',
#  'OMNIPOTENCE',
#  'PHILOSOPHER',
#  'POWER',
#  'PRAYER',
#  'PREJUDICE',
#  'RARE',
#  'REASON',
#  'RELIGION',
#  'SECT',
#  'SELF-ESTEEM',
#  'SOUL',
#  'STATE',
#  'SUPERSTITION',
#  'TEAR',
#  'THEIST',
#  'TOLERANCE',
#  'TRUTH',
#  'TYRANNY',
#  'VIRTUE',
#  'WHY',
#  'ANCIENT',
#  'MODERN',
#  'BOULEVART',
#  'CIVIL LAW',
#  'COMMON SENSE',
#  'EVENT',
#  'DEVOUT',
#  'ECCLESIAST',
#  'THEATRE',
#  'FALSE MIND',
#  'FINAL CAUSES',
#  'IMPIOUS',
#  'JOAN OF ARC',
#  'LIMITS OF THE HUMAN MIND',
#  'LOCAL CRIMES',
#  'GENERAL REFLECTION ON MAN',
#  'IRON MASK',
#  'MEN OF LETTERS',
#  'MILTON',
#  'NATURAL LAW',
#  'NEW NOVELTIES',
#  'PRECIS'])
# terms=[w.lower() for w in terms]
# print len(terms)
# terms

In [ ]:
# with open('terms.pkl', 'wb') as f:
#     pickle.dump(terms, f)

In [ ]:
with open('terms.pkl', 'rb') as f:
     terms = pickle.load(f)
print len(terms)
terms

### List of References (manually collected)

In [ ]:
references=[
("LOCAL CRIMES","ADULTERY"),
("ADULTERY","ADVOCATE"),
("TOLERANCE","ANCIENTS AND MODERNS"),
    ("LOVE","ANIMALS"),
    ("NATURE","ANIMALS"),
    ("POWER","ANIMALS"),
    ("OMNIPOTENCE","ANIMALS"),
    ("SOUL","ANIMALS"),
    ("WHY?","ANIMALS"),
    ("ARTS","ANIMALS"),
    ("ATHEISM","ANIMALS"),
    ("BOURGES","ANIMALS"),
    ("BRAHMINS","ANIMALS"),
    ("CLIMATE","ANIMALS"),
    ("CONCATENATION OF EVENTS","ANIMALS"),
    ("CONTRADICTIONS","ANIMALS"),
    ("DESTINY","ANIMALS"),
    ("EQUALITY","ANIMALS"),
    ("FALSE MINDS","ANIMALS"),
    ("FREE-WILL","ANIMALS"),
    ("IGNORANCE","ANIMALS"),
    ("LAWS","ANIMALS"),
    ("LIMITS OF THE HUMAN MIND","ANIMALS"),
    ("LUXURY","ANTIQUITY"),
    ("PHILOSOPHER","ANTIQUITY"),
    ("POWER","ANTIQUITY"),
    ("OMNIPOTENCE","ANTIQUITY"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","ANTIQUITY"),
    ("RELIGION","ANTIQUITY"),
    ("SOUL","ANTIQUITY"),
    ("WHY?","ANTIQUITY"),
    ("ANCIENTS AND MODERNS","ANTIQUITY"),
    ("ARTS","ANTIQUITY"),
    ("ATHEISM","ANTIQUITY"),
    ("BOURGES","ANTIQUITY"),
    ("CORN","ANTIQUITY"),
    ("DESTINY","ANTIQUITY"),
    ("EMBLEM","ANTIQUITY"),
    ("ENVY","ANTIQUITY"),
    ("EXPIATION","ANTIQUITY"),
    ("HISTORY","ANTIQUITY"),
    ("LIBRARY","ANTIQUITY"),
    ("LUXURY","ARTS"),
    ("MEN OF LETTERS","ARTS"),
    ("MOHAMMEDANS","ARTS"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","ARTS"),
    ("RARE","ARTS"),
    ("SOUL","ARTS"),
    ("AUTHORS","ARTS"),
    ("ENVY","ARTS"),
    ("EXTREME","ARTS"),
    ("FINAL CAUSES","ARTS"),
    ("HISTORY","ARTS"),
    ("LANGUAGES","ARTS"),
    ("ENVY","ATHEISM"),
    ("THE ECCLESIASTICAL MINISTRY","AUTHORITY"),
    ("EQUALITY","AUTHORITY"),
    ("JOAN OF ARC","AUTHORITY"),
    ("ANCIENTS AND MODERNS","AUTHORS"),
    ("ASTROLOGY","AUTHORS"),
    ("BOOKS","AUTHORS"),
    ("FRENCH","AUTHORS"),
    ("KISSING","AUTHORS"),
    ("LANGUAGES","AUTHORS"),
    ("LIBRARY","AUTHORS"),
    ("MASTER","AUTHORS"),
    ("SOUL","AUTHORS"),
    ("MILTON, ON THE REPROACH OF PLAGIARISM AGAINST","BANISHMENT"),
    ("LOVE","BEAUTY"),
    ("RARE","BEAUTY"),
    ("ATHEISM","BEAUTY"),
    ("ON THE ENGLISH THEATRE","BEAUTY"),
    ("FREE-WILL","BEAUTY"),
    ("SOUL","BISHOP"),
    ("SUPERSTITION","BISHOP"),
    ("ADULTERY","BISHOP"),
    ("CROMWELL","BISHOP"),
    ("EXPIATION","BISHOP"),
    ("JOAN OF ARC","BISHOP"),
    ("CLIMATE","BRAHMINS"),
    ("EZOURVEIDAM","BRAHMINS"),
    ("LIBRARY","BRAHMINS"),
    ("AUTHORITY","BOOKS"),
    ("AUTHORS","BOOKS"),
    ("BISHOP","BOOKS"),
    ("BRAHMINS","BOOKS"),
    ("CONCATENATION OF EVENTS","BOOKS"),
    ("DESTINY","BOOKS"),
    ("THE ECCLESIASTICAL MINISTRY","BOOKS"),
    ("EMBLEM","BOOKS"),
    ("EZOURVEIDAM","BOOKS"),
    ("FRENCH","BOOKS"),
    ("HISTORY","BOOKS"),
    ("KISSING","BOOKS"),
    ("LANGUAGES","BOOKS"),
    ("LIBERTY","BOOKS"),
    ("LIBRARY","BOOKS"),
    ("NATURAL LAW","BOOKS"),
    ("NEW NOVELTIES","BOOKS"),
    ("SECT","BOOKS"),
    ("TRUTH","BOOKS"),
    ("ON THE ENGLISH THEATRE","CHARACTER"),
    ("ENVY","CHARACTER"),
    ("EXTREME","CHARACTER"),
    ("HISTORY","CHARACTER"),
    ("LANGUAGES","CHARACTER"),
    ("PHILOSOPHER","CHARACTER"),
    ("SECT","CHARACTER"),
    ("SOUL","CHARACTER"),
    ("STATES","CHARACTER"),
    ("GOVERNMENTS","CHARACTER"),
    ("NECESSARY","CHARLATAN"),
    ("PREJUDICES","CHARLATAN"),
    ("ADULTERY","CIVIL LAWS"),
    ("ANCIENTS AND MODERNS","CLIMATE"),
    ("ANTIQUITY","CLIMATE"),
    ("CORN","CLIMATE"),
    ("EQUALITY","CLIMATE"),
    ("STATES","CLIMATE"),
    ("GOVERNMENTS","CLIMATE"),
    ("ADULTERY","COMMON SENSE"),
    ("HISTORY","COMMON SENSE"),
    ("JOAN OF ARC","COMMON SENSE"),
    ("BRAHMINS","CONTRADICTIONS"),
    ("THE ECCLESIASTICAL MINISTRY","CONTRADICTIONS"),
    ("SECT","CONTRADICTIONS"),
    ("LAWS","CORN"),
    ("LIMITS OF THE HUMAN MIND","CORN"),
    ("LUXURY","CORN"),
    ("NATURE","CORN"),
    ("REASON","CORN"),
    ("CHARLATAN","CROMWELL"),
    ("CONTRADICTIONS","CROMWELL"),
    ("DESTINY","CROMWELL"),
    ("NATURAL LAW","CROMWELL"),
    ("AUTHORITY","CUSTOMS"),
    ("CLIMATE","CUSTOMS"),
    ("DESTINY","CUSTOMS"),
    ("EMBLEM","CUSTOMS"),
    ("HISTORY","CUSTOMS"),
    ("NATURAL LAW","CUSTOMS"),
    ("FATHERLAND","DEMOCRACY"),
    ("LAWS","DEMOCRACY"),
    ("CONCATENATION OF EVENTS","DESTINY"),
    ("CROMWELL","DESTINY"),
    ("MASTER","DESTINY"),
    ("CROMWELL","DEVOUT"),
    ("BRAHMINS","EMBLEM"),
    ("DEMOCRACY","EQUALITY"),
    ("EXTREME","EQUALITY"),
    ("LUXURY","EQUALITY"),
    ("ATHEISM","EXTREME"),
    ("BOOKS","EXTREME"),
    ("SOUL","EXTREME"),
    ("BRAHMINS","EZOURVEIDAM"),
    ("BRAHMINS","FAITH"),
    ("DEVOUT","FAITH"),
    ("HISTORY","FAITH"),
    ("JOAN OF ARC","FAITH"),
    ("KISSING","FAITH"),
    ("POWER","FAITH"),
    ("OMNIPOTENCE","FAITH"),
    ("SOUL","FAITH"),
    ("THEIST","FAITH"),
    ("VIRTUE","FAITH"),
    ("MARRIAGE","FATHERLAND"),
    ("RELIGION","FATHERLAND"),
    ("ADULTERY","FATHERLAND"),
    ("BANISHMENT","FATHERLAND"),
    ("TEARS","FINAL CAUSES"),
    ("ATHEISM","FINAL CAUSES"),
    ("MILTON, ON THE REPROACH OF PLAGIARISM AGAINST","FRAUD"),
    ("TEARS","FRAUD"),
    ("LIBERTY","FREE-WILL"),
    ("ADULTERY","FRENCH"),
    ("ANIMALS","FRENCH"),
    ("BANISHMENT","FRENCH"),
    ("BEAUTY","FRENCH"),
    ("BOOKS","FRENCH"),
    ("ON THE ENGLISH THEATRE","FRENCH"),
    ("EZOURVEIDAM","FRENCH"),
    ("HISTORY","FRENCH"),
    ("JOAN OF ARC","FRENCH"),
    ("KISSING","FRENCH"),
    ("LANGUAGES","FRENCH"),
    ("MEN OF LETTERS","FRENCH"),
    ("SOUL","FRENCH"),
    ("ANIMALS","FRIENDSHIP"),
    ("ATHEISM","FRIENDSHIP"),
    ("CROMWELL","FRIENDSHIP"),
    ("FINAL CAUSES","FRIENDSHIP"),
    ("LOVE","FRIENDSHIP"),
    ("ADULTERY","GOD"),
    ("ANCIENTS AND MODERNS","GOD"),
    ("ATHEISM","GOD"),
    ("BRAHMINS","GOD"),
    ("CHARLATAN","GOD"),
    ("CLIMATE","GOD"),
    ("CONCATENATION OF EVENTS","GOD"),
    ("CROMWELL","GOD"),
    ("CUSTOMS","GOD"),
    ("DEMOCRACY","GOD"),
    ("DESTINY","GOD"),
    ("THE ECCLESIASTICAL MINISTRY","GOD"),
    ("EMBLEM","GOD"),
    ("EXPIATION","GOD"),
    ("FAITH","GOD"),
    ("FALSE MINDS","GOD"),
    ("FINAL CAUSES","GOD"),
    ("FRAUD","GOD"),
    ("HISTORY","GOD"),
    ("IGNORANCE","GOD"),
    ("THE IMPIOUS","GOD"),
    ("JOAN OF ARC","GOD"),
    ("KISSING","GOD"),
    ("LANGUAGES","GOD"),
    ("LIBERTY","GOD"),
    ("LUXURY","GOD"),
    ("MASTER","GOD"),
    ("MEN OF LETTERS","GOD"),
    ("MILTON, ON THE REPROACH OF PLAGIARISM AGAINST","GOD"),
    ("NAKEDNESS","GOD"),
    ("NECESSARY","GOD"),
    ("PHILOSOPHER","GOD"),
    ("POWER","GOD"),
    ("OMNIPOTENCE","GOD"),
    ("PRAYERS","GOD"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","GOD"),
    ("PREJUDICES","GOD"),
    ("RELIGION","GOD"),
    ("SECT","GOD"),
    ("SOUL","GOD"),
    ("THEIST","GOD"),
    ("TOLERANCE","GOD"),
    ("VIRTUE","GOD"),
    ("WHY?","GOD"),
    ("ADULTERY","HISTORY"),
    ("ANTIQUITY","HISTORY"),
    ("AUTHORS","HISTORY"),
    ("BOOKS","HISTORY"),
    ("BRAHMINS","HISTORY"),
    ("CHARLATAN","HISTORY"),
    ("CONTRADICTIONS","HISTORY"),
    ("CORN","HISTORY"),
    ("EXPIATION","HISTORY"),
    ("EXTREME","HISTORY"),
    ("JOAN OF ARC","HISTORY"),
    ("MAN IN THE IRON MASK","HISTORY"),
    ("SOUL","HISTORY"),
    ("ANCIENTS AND MODERNS","IGNORANCE"),
    ("DEVOUT","IGNORANCE"),
    ("EXPIATION","IGNORANCE"),
    ("HISTORY","IGNORANCE"),
    ("LANGUAGES","IGNORANCE"),
    ("SOUL","IGNORANCE"),
    ("TOLERANCE","IGNORANCE"),
    ("LOVE","KISSING"),
    ("NATURE","KISSING"),
    ("FRENCH","LANGUAGES"),
    ("MARRIAGE","LANGUAGES"),
    ("SOUL","LANGUAGES"),
    ("ADULTERY","LAWS"),
    ("ADVOCATE","LAWS"),
    ("ANCIENTS AND MODERNS","LAWS"),
    ("ANTIQUITY","LAWS"),
    ("ATHEISM","LAWS"),
    ("BANKRUPTCY","LAWS"),
    ("BEAUTY","LAWS"),
    ("CHARLATAN","LAWS"),
    ("CIVIL LAWS","LAWS"),
    ("CLIMATE","LAWS"),
    ("CONTRADICTIONS","LAWS"),
    ("DESTINY","LAWS"),
    ("EXPIATION","LAWS"),
    ("EXTREME","LAWS"),
    ("FATHERLAND","LAWS"),
    ("FREE-WILL","LAWS"),
    ("FRENCH","LAWS"),
    ("HISTORY","LAWS"),
    ("LANGUAGES","LAWS"),
    ("LUXURY","LAWS"),
    ("MASTER","LAWS"),
    ("NATURAL LAW","LAWS"),
    ("NATURE","LAWS"),
    ("PHILOSOPHER","LAWS"),
    ("POWER","LAWS"),
    ("OMNIPOTENCE","LAWS"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","LAWS"),
    ("RELIGION","LAWS"),
    ("STATES","LAWS"),
    ("GOVERNMENTS","LAWS"),
    ("TYRANNY","LAWS"),
    ("BOOKS","LIBERTY"),
    ("CHARLATAN","LIBERTY"),
    ("DEMOCRACY","LIBERTY"),
    ("DESTINY","LIBERTY"),
    ("THE ECCLESIASTICAL MINISTRY","LIBERTY"),
    ("FREE-WILL","LIBERTY"),
    ("FRENCH","LIBERTY"),
    ("HELVETIA","LIBERTY"),
    ("KISSING","LIBERTY"),
    ("LUXURY","LIBERTY"),
    ("NECESSARY","LIBERTY"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","LIBERTY"),
    ("SOUL","LIBERTY"),
    ("TOLERANCE","LIBERTY"),
    ("EZOURVEIDAM","LIBRARY"), 
    ("GOD","LIBRARY"),
    ("ANCIENTS AND MODERNS","LOVE"),
    ("ANIMALS","LOVE"),
    ("ATHEISM","LOVE"),
    ("CLIMATE","LOVE"),
    ("CROMWELL","LOVE"),
    ("DESTINY","LOVE"),
    ("FATHERLAND","LOVE"),
    ("FRAUD","LOVE"),
    ("FREE-WILL","LOVE"),
    ("KISSING","LOVE"),
    ("LANGUAGES","LOVE"),
    ("LAWS","LOVE"),
    ("LIBERTY","LOVE"),
    ("PHILOSOPHER","LOVE"),
    ("PREJUDICES","LOVE"),
    ("RELIGION","LOVE"),
    ("SELF-ESTEEM","LOVE"),
    ("SOUL","LOVE"),
    ("VIRTUE","LOVE"),
    ("MILTON, ON THE REPROACH OF PLAGIARISM AGAINST","LUXURY"),
    ("ARTS","LUXURY"),
    ("ADULTERY","MARRIAGE"),
    ("CONCATENATION OF EVENTS","MARRIAGE"),
    ("EMBLEM","MARRIAGE"),
    ("FRIENDSHIP","MARRIAGE"),
    ("MAN IN THE IRON MASK","MARRIAGE"),
    ("MASTER","MARRIAGE"),
    ("NAKEDNESS","MARRIAGE"),
    ("REASON","MARRIAGE"),
    ("ADULTERY","MASTER"),
    ("ANIMALS","MASTER"),
    ("ASTROLOGY","MASTER"),
    ("ATHEISM","MASTER"),
    ("AUTHORITY","MASTER"),
    ("BISHOP","MASTER"),
    ("CHARACTER","MASTER"),
    ("CHARLATAN","MASTER"),
    ("CONCATENATION OF EVENTS","MASTER"),
    ("CROMWELL","MASTER"),
    ("DEMOCRACY","MASTER"),
    ("DESTINY","MASTER"),
    ("DEVOUT","MASTER"),
    ("ON THE ENGLISH THEATRE","MASTER"),
    ("EQUALITY","MASTER"),
    ("FALSE MINDS","MASTER"),
    ("FRAUD","MASTER"),
    ("FREE-WILL","MASTER"),
    ("GOD","MASTER"),
    ("THE IMPIOUS","MASTER"),
    ("JOAN OF ARC","MASTER"),
    ("LAWS","MASTER"),
    ("LIBRARY","MASTER"),
    ("MAN IN THE IRON MASK","MASTER"),
    ("POWER","MASTER"),
    ("OMNIPOTENCE","MASTER"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","MASTER"),
    ("RELIGION","MASTER"),
    ("SOUL","MASTER"),
    ("TOLERANCE","MASTER"),
    ("VIRTUE","MASTER"),
    ("WHY?","MASTER"),
    ("MAN IN THE IRON MASK","MEN OF LETTERS"),
    ("NAKEDNESS","MOHAMMEDANS"),
    ("RELIGION","MOHAMMEDANS"),
    ("CLIMATE","MOHAMMEDANS"),
    ("ANTIQUITY","MOUNTAIN"),
    ("ARTS","MOUNTAIN"),
    ("BRAHMINS","MOUNTAIN"),
    ("CHARLATAN","MOUNTAIN"),
    ("CONTRADICTIONS","MOUNTAIN"),
    ("DEMOCRACY","MOUNTAIN"),
    ("EXTREME","MOUNTAIN"),
    ("FINAL CAUSES","MOUNTAIN"),
    ("HELVETIA","MOUNTAIN"),
    ("THE IMPIOUS","MOUNTAIN"),
    ("LOCAL CRIMES","MOUNTAIN"),
    ("NATURE","MOUNTAIN"),
    ("STATES","MOUNTAIN"),
    ("GOVERNMENTS","MOUNTAIN"),
    ("THE ECCLESIASTICAL MINISTRY","NATURAL LAW"),
    ("LAWS","NATURAL LAW"),
    ("ANCIENTS AND MODERNS","NATURE"),
    ("ANIMALS","NATURE"),
    ("ATHEISM","NATURE"),
    ("BRAHMINS","NATURE"),
    ("CHARACTER","NATURE"),
    ("CONCATENATION OF EVENTS","NATURE"),
    ("CONTRADICTIONS","NATURE"),
    ("DEMOCRACY","NATURE"),
    ("DESTINY","NATURE"),
    ("THE ECCLESIASTICAL MINISTRY","NATURE"),
    ("EMBLEM","NATURE"),
    ("ON THE ENGLISH THEATRE","NATURE"),
    ("EQUALITY","NATURE"),
    ("EXTREME","NATURE"),
    ("FINAL CAUSES","NATURE"),
    ("FRAUD","NATURE"),
    ("FREE-WILL","NATURE"),
    ("GOD","NATURE"),
    ("HELVETIA","NATURE"),
    ("IGNORANCE","NATURE"),
    ("THE IMPIOUS","NATURE"),
    ("KISSING","NATURE"),
    ("LAWS","NATURE"),
    ("LIMITS OF THE HUMAN MIND","NATURE"),
    ("LOVE","NATURE"),
    ("LUXURY","NATURE"),
    ("MASTER","NATURE"),
    ("METAMORPHOSIS","NATURE"),
    ("METEMPSYCHOSIS","NATURE"),
    ("NATURAL LAW","NATURE"),
    ("NECESSARY","NATURE"),
    ("PHILOSOPHER","NATURE"),
    ("POWER","NATURE"),
    ("OMNIPOTENCE","NATURE"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","NATURE"),
    ("PREJUDICES","NATURE"),
    ("RARE","NATURE"),
    ("RELIGION","NATURE"),
    ("SOUL","NATURE"),
    ("TEARS","NATURE"),
    ("TOLERANCE","NATURE"),
    ("SOUL","OMNIPOTENCE"),
    ("SUPERSTITION","PRAYERS"),
    ("CROMWELL","PRAYERS"),
    ("THE ECCLESIASTICAL MINISTRY","PRAYERS"),
    ("STATES","PREJUDICES"),
    ("GOVERNMENTS","PREJUDICES"),
    ("SUPERSTITION","PREJUDICES"),
    ("COMMON SENSE","PREJUDICES"),
    ("DESTINY","PREJUDICES"),
    ("ARTS","RARE"),
    ("AUTHORS","RARE"),
    ("BOOKS","RARE"),
    ("CLIMATE","RARE"),
    ("COMMON SENSE","RARE"),
    ("FATHERLAND","RARE"),
    ("HELVETIA","RARE"),
    ("HISTORY","RARE"),
    ("LAWS","RARE"),
    ("MARRIAGE","RARE"),
    ("NATURE","RARE"),
    ("RELIGION","RARE"),
    ("SOUL","RARE"),
    ("STATES","RARE"),
    ("GOVERNMENTS","RARE"),
    ("ASTROLOGY","REASON"),
    ("ATHEISM","REASON"),
    ("AUTHORITY","REASON"),
    ("BANISHMENT","REASON"),
    ("BANKRUPTCY","REASON"),
    ("BEAUTY","REASON"),
    ("BOURGES","REASON"),
    ("BRAHMINS","REASON"),
    ("CHARLATAN","REASON"),
    ("COMMON SENSE","REASON"),
    ("CONCATENATION OF EVENTS","REASON"),
    ("CROMWELL","REASON"),
    ("DESTINY","REASON"),
    ("THE ECCLESIASTICAL MINISTRY","REASON"),
    ("ON THE ENGLISH THEATRE","REASON"),
    ("ENVY","REASON"),
    ("EQUALITY","REASON"),
    ("FALSE MINDS","REASON"),
    ("FATHERLAND","REASON"),
    ("FRAUD","REASON"),
    ("FREE-WILL","REASON"),
    ("FRENCH","REASON"),
    ("HELVETIA","REASON"),
    ("IGNORANCE","REASON"),
    ("THE IMPIOUS","REASON"),
    ("JOAN OF ARC","REASON"),
    ("LANGUAGES","REASON"),
    ("LAWS","REASON"),
    ("LIBERTY","REASON"),
    ("LUXURY","REASON"),
    ("GENERAL REFLECTION ON MAN","REASON"),
    ("MAN IN THE IRON MASK","REASON"),
    ("MARRIAGE","REASON"),
    ("MASTER","REASON"),
    ("NATURE","REASON"),
    ("NECESSARY","REASON"),
    ("POWER","REASON"),
    ("OMNIPOTENCE","REASON"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","REASON"),
    ("PREJUDICES","REASON"),
    ("RARE","REASON"),
    ("SECT","REASON"),
    ("SOUL","REASON"),
    ("STATES","REASON"),
    ("GOVERNMENTS","REASON"),
    ("SUPERSTITION","REASON"),
    ("TOLERANCE","REASON"),
    ("TRUTH","REASON"),
    ("ATHEISM","RELIGION"),
    ("AUTHORS","RELIGION"),
    ("CHARACTER","RELIGION"),
    ("CLIMATE","RELIGION"),
    ("CROMWELL","RELIGION"),
    ("CUSTOMS","RELIGION"),
    ("THE ECCLESIASTICAL MINISTRY","RELIGION"),
    ("EXPIATION","RELIGION"),
    ("FRAUD","RELIGION"),
    ("LANGUAGES","RELIGION"),
    ("MOHAMMEDANS","RELIGION"),
    ("NATURAL LAW","RELIGION"),
    ("NECESSARY","RELIGION"),
    ("PRAYERS","RELIGION"),
    ("PREJUDICES","RELIGION"),
    ("REASON","RELIGION"),
    ("SECT","RELIGION"),
    ("SOUL","RELIGION"),
    ("STATES","RELIGION"),
    ("GOVERNMENTS","RELIGION"),
    ("SUPERSTITION","RELIGION"),
    ("THEIST","RELIGION"),
    ("TOLERANCE","RELIGION"),
    ("ADULTERY","SECT"),
    ("ATHEISM","SECT"),
    ("CHARLATAN","SECT"),
    ("DESTINY","SECT"),
    ("KISSING","SECT"),
    ("NAKEDNESS","SECT"),
    ("NECESSARY","SECT"),
    ("PHILOSOPHER","SECT"),
    ("POWER","SECT"),
    ("OMNIPOTENCE","SECT"),
    ("SOUL","SECT"),
    ("SUPERSTITION","SECT"),
    ("THEIST","SECT"),
    ("TOLERANCE","SECT"),
    ("WHY","SECT"),
    ("ANCIENTS AND MODERNS","SELF-ESTEEM"),
    ("AUTHORS","SELF-ESTEEM"),
    ("LAWS","SELF-ESTEEM"),
    ("POWER","SELF-ESTEEM"),
    ("OMNIPOTENCE","SELF-ESTEEM"),
    ("ANIMALS","SOUL"),
    ("ATHEISM","SOUL"),
    ("CHARACTER","SOUL"),
    ("CHARLATAN","SOUL"),
    ("CLIMATE","SOUL"),
    ("COMMON SENSE","SOUL"),
    ("THE ECCLESIASTICAL MINISTRY","SOUL"),
    ("EXPIATION","SOUL"),
    ("FALSE MINDS","SOUL"),
    ("FREE-WILL","SOUL"),
    ("FRIENDSHIP","SOUL"),
    ("IGNORANCE","SOUL"),
    ("LANGUAGES","SOUL"),
    ("LUXURY","SOUL"),
    ("GENERAL REFLECTION ON MAN","SOUL"),
    ("MAN IN THE IRON MASK","SOUL"),
    ("MARRIAGE","SOUL"),
    ("MEN OF LETTERS","SOUL"),
    ("METAMORPHOSIS","SOUL"),
    ("PHILOSOPHER","SOUL"),
    ("TEARS","SOUL"),
    ("WHY?","SOUL"),
    ("ADULTERY","STATES"),
    ("BANKRUPTCY","STATES"),
    ("CONCATENATION OF EVENTS","STATES"),
    ("CONTRADICTIONS","STATES"),
    ("HISTORY","STATES"),
    ("JOAN OF ARC","STATES"),
    ("LAWS","STATES"),
    ("ANCIENTS AND MODERNS","SUPERSTITION"),
    ("ATHEISM","SUPERSTITION"),
    ("CUSTOMS","SUPERSTITION"),
    ("EXTREME","SUPERSTITION"),
    ("FRAUD","SUPERSTITION"),
    ("JOAN OF ARC","SUPERSTITION"),
    ("NAKEDNESS","SUPERSTITION"),
    ("RELIGION","SUPERSTITION"),
    ("TOLERANCE","SUPERSTITION"),
    ("JOAN OF ARC","TEARS"),
    ("ATHEISM","THEIST"),
    ("WHY?","THEIST"),
    ("ANCIENTS AND MODERNS","TRUTH"),
    ("ARTS","TRUTH"),
    ("ATHEISM","TRUTH"),
    ("CHARLATAN","TRUTH"),
    ("CONCATENATION OF EVENTS","TRUTH"),
    ("DESTINY","TRUTH"),
    ("THE ECCLESIASTICAL MINISTRY","TRUTH"),
    ("EMBLEM","TRUTH"),
    ("EXTREME","TRUTH"),
    ("FATHERLAND","TRUTH"),
    ("FRAUD","TRUTH"),
    ("HISTORY","TRUTH"),
    ("JOAN OF ARC","TRUTH"),
    ("MAN IN THE IRON MASK","TRUTH"),
    ("MEN OF LETTERS","TRUTH"),
    ("MILTON, ON THE REPROACH OF PLAGIARISM AGAINST","TRUTH"),
    ("MOHAMMEDANS","TRUTH"),
    ("NATURE","TRUTH"),
    ("NECESSARY","TRUTH"),
    ("PHILOSOPHER","TRUTH"),
    ("PRÉCIS OF ANCIENT PHILOSOPHY","TRUTH"),
    ("PREJUDICES","TRUTH"),
    ("SECT","TRUTH"),
    ("VIRTUE","TRUTH"),
    ("WHY?","TRUTH"),
    ("CONTRADICTIONS","TYRANNY"),
    ("DEMOCRACY","TYRANNY"),
    ("ADULTERY","VIRTUE"),
    ("ATHEISM","VIRTUE"),
    ("BANKRUPTCY","VIRTUE"),
    ("BEAUTY","VIRTUE"),
    ("CHARLATAN","VIRTUE"),
    ("THE ECCLESIASTICAL MINISTRY","VIRTUE"),
    ("EMBLEM","VIRTUE"),
    ("EXPIATION","VIRTUE"),
    ("FRAUD","VIRTUE"),
    ("FREE-WILL","VIRTUE"),
    ("LAWS","VIRTUE"),
    ("LUXURY","VIRTUE"),
    ("PHILOSOPHER","VIRTUE"),
    ("PREJUDICES","VIRTUE"),
    ("RELIGION","VIRTUE"),
    ("SOUL","VIRTUE"),
    ("STATES","VIRTUE"),
    ("GOVERNMENTS","VIRTUE"),
    ("TEARS","VIRTUE"),
    ("TOLERANCE","VIRTUE"),
    ("SECT","WHY?"),
    ("TEARS","WHY?"),
    ("TOLERANCE","WHY?"),
    ("FREE-WILL","WHY?")
    
]

In [ ]:
len(references)

### Load data

In [ ]:
filename = 'Voltaire_PhilosophicalDictionary_mod1.txt' # Insert in quotes the name of your txt file, e.g. "PeterPan.txt"
titlename = "Voltaire's Philosophical Dictionary"   # Insert in quotes the name of the novel (or book), e.g., "Peter Pan"

f = codecs.open(filename, "r", encoding="utf-8").readlines()

num_lines = 0
num_words = 0
num_chars = 0
for line in f:
    words = line.split()
    num_lines += 1
    num_words += len(words)
    num_chars += len(line)
print "%s has number of words = %i (and number of characters/symbols = %i)" %(titlename,num_words,num_chars)

In [ ]:
text=", ".join(f).encode('ascii','ignore').replace('\n',' ')
print len(text), type(text)
text